In [ ]:
# Imports

import sys
import os
import scipy

import torch
import onnxruntime as ort

import matplotlib.pyplot as plt

import yaml
from tqdm import tqdm

sys.path.append("../openWakeWord")
import openwakeword
from openwakeword.utils import bulk_predict
from openwakeword import Model

from IPython.display import Audio
from pathlib import Path

# Load Config for the Model

In [ ]:
# Load the YAML config file

with open('../wakeword_training_configs/angela.yml', 'r') as file:
    config = yaml.safe_load(file)

config["model_name"] = 'angela'
model_path = os.path.join("../trained_models", config["model_name"] + ".onnx")

# Load the model

In [ ]:
# Instantiate the model(s)
model = Model(
    wakeword_models=[model_path],
    inference_framework = 'onnx',
)


# Utility Functions

In [ ]:
def get_prediction_times(oww_instance, audio_file, score_threshold, cooldown = 1):
    """Helper function that takes in an audio file and returns the prediction times for the wakeword model.

    The times correspond to approximately the end of the spoken word in the audio file
    
    Args:
        oww_instance (openwakeword.Model): OpenWakeWord model instance, with the wakeword model loaded
        audio_file (str): Path to the audio file
        score_threshold (float): The score threshold for the wakeword model
        cooldown (int): The number of seconds to wait after a prediction before returning another prediction

    Returns:
        tuple: Tuple of prediction times and scores (each a list)
    """
    # Load the file
    sr, dat = scipy.io.wavfile.read(audio_file)

    # Convert to 16-bit PCM, as needed
    if dat.dtype == 'float32' and dat.max() <= 1.0:
        dat = (dat*32767).astype('int16')

    # Get the predictions
    predictions = oww_instance.predict_clip(dat, padding = 0)
    mdl_name = list(predictions[0].keys())[0]

    # Get the scores and aligned times
    scores = [i[mdl_name] for i in predictions]
    times = [0.08*(i+1) for i in range(len(scores))]  # default of 80 ms per prediction frame

    # Find the first score that is above the threshold
    prediction_times = []
    last_prediction_time = 0
    for i, score in enumerate(scores):
        if score > score_threshold:
            if times[i] - last_prediction_time >= cooldown:
                last_prediction_time = times[i]
                prediction_times.append(times[i])

    return prediction_times, scores


# Get Predictions on the test data

## Positive Clips (with wakeword)

In [ ]:
# Positive test clips (from Polyn AI)

positive_test_clips = [str(i) for i in Path("../data/train_angela_20250206/train/angela").glob("*.wav")]

len(positive_test_clips)

In [ ]:
# Estimate the recall on positive recordings

tp = 0
fn = 0
score_threshold = 0.5
missed = []

for clip in tqdm(positive_test_clips):
    predictions = model.predict_clip(clip)
    scores = [i[config["model_name"]] for i in predictions]
    if max(scores) > score_threshold:
        tp += 1
    else:
        fn += 1
        missed.append(clip)

# Calculate recall
recall = tp / (tp + fn)
recall

In [ ]:
# listen to missed clips

display(Audio(missed[0]))

## Positive Clips with background noise at 9 dB SNR

In [ ]:
# Get times for predictions

background_clip = "../data/data_examples.zip.001_FILES/long/CChWECeZ_dRn8VaOm_noise=110nsroomofficelevel=0dB_level=-6.0dB_snr=9.0dB.wav"

# Get the prediction times for the audio file
times, scores = get_prediction_times(model,
                                    "..//data/data_examples.zip.001_FILES/long/CChWECeZ_dRn8VaOm_noise=110nsroomofficelevel=0dB_level=-6.0dB_snr=9.0dB.wav",
                                    score_threshold = 0.5
)


In [ ]:
# Compare times to corresponding segments in the actual audio file

# Load the audio file
sr, audio = scipy.io.wavfile.read("..//data/data_examples.zip.001_FILES/long/CChWECeZ_dRn8VaOm_noise=110nsroomofficelevel=0dB_level=-6.0dB_snr=9.0dB.wav")

# Select the audio to play (1.5 seconds before the prediction time)
prediction_n = 7
start_time = times[prediction_n] - 1.5
start_idx = int(start_time*sr)
end_idx = int(times[prediction_n]*sr)
audio_to_play = audio[start_idx:end_idx]

# Play the audio
Audio(audio_to_play, rate=sr)

In [ ]:
# Time of target wakewords in the example validation clip (estimated in seconds, from manually listening to the clip)

def convert_to_seconds(time_str):
    if isinstance(time_str, int):
        return time_str
    else:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds

wakeword_times = [
    4, 9, 17, 19, 22, 25, 28, 44, 49, 54, 59, "1:02", "1:05", "1:08", "1:11", "1:18", "1:21", "1:23", "1:26", "1:29", "1:31", "1:39",
    "1:49", "1:51", "1:54", "2:01", "2:04", "2:07", "2:09", "2:11", "2:19", "2:32", "2:44", "2:49", "3:04", "3:06", "3:09", "3:11", "3:14",
    "3:19", "3:22", "3:30", "3:34", "3:39", "3:52", "3:57", "4:00", "4:04", "4:07", "4:10"
]

wakeword_times = [convert_to_seconds(t) for t in wakeword_times]


In [ ]:
# Make function check for predictions within a certain time window of the target wakeword

def check_for_true_positives(predictions_times, target_times, time_window=2):
    """
    Check if predictions occur within a time window of target wakeword times
    
    Args:
        predictions_times (list): List of times when positive predictions occurred
        target_times (list): List of actual wakeword occurrence times
        time_window (float): Time window in seconds to consider a prediction as correct
        
    Returns:
        tuple: (true_positives, false_positives)
    """
    true_positives = 0
    false_positives = []
    false_negatives = []
    
    # For each target time, check if there's a prediction within the time window
    for target in target_times:
        found_match = False
        for pred_time in predictions_times:
            if abs(pred_time - target) <= time_window:
                found_match = True
                break
        if found_match:
            true_positives += 1

        # Save positions of false negatives
        else:
            false_negatives.append(target)
            
    # Count predictions that don't match any target times as false positives
    for pred_time in predictions_times:
        is_false_positive = True
        for target in target_times:
            if abs(pred_time - target) <= time_window:
                is_false_positive = False
                break
        if is_false_positive:
            false_positives.append(pred_time)
    
    return true_positives, false_positives, false_negatives

# Test the function
true_pos, false_pos, false_neg = check_for_true_positives(times, wakeword_times)
print(f"True Positives: {true_pos}/{len(wakeword_times)} ({true_pos/len(wakeword_times)*100:.2f}%)")
print(f"False Positives: {len(false_pos)}")

In [ ]:
# Plot the predictions and positions of the target wakewords

times = [i*.08 for i in range(len(scores))]

plt.figure(figsize=(20, 5))
plt.plot(times, scores)

for t in wakeword_times:
    plt.axvline(x=[t], color='r', linestyle='--')

# plt.xlim(180, 240)

## Negative clips (to estimate false activation rate)

In [ ]:
# Load negative clips (for false activation testing)
# from the far_train_20250205.tar.gz file provided by Polyn AI

negative_test_clips = [str(i) for i in Path("/home/dscripka/dscripkaDrive_nvme_fast/polynai_project/train/").glob("**/*.wav")]

len(negative_test_clips)

In [ ]:
# Get predictions from test files

predictions = bulk_predict(
    file_paths = negative_test_clips[0:24],
    wakeword_models=[model_path],
    ncpu=8 # set this to the number of CPU cores you have available
)

# Combine all scores
all_scores = []
for key in predictions.keys():
    all_scores += [i[config["model_name"]] for i in predictions[key]]

# Calculate the total time in hours
total_time_hours = len(all_scores)*.08/3600

print(f"Total time: {total_time_hours} hours")

In [ ]:
# Estimate the false positive rate per hour

from openwakeword.metrics import get_false_positives

num_false_positives = get_false_positives(all_scores, threshold=0.5, grouping_window=20)
print("False positives per hour: ", num_false_positives/total_time_hours)